In [1]:
nd = require('../src/numb/ndarray');
Numb = require('../src/numb/numb');
Autograd = require('../src/numb/autograd/autograd');
Ops  = require('../src/numb/operators');
GradOps = require('../src/numb/autograd/gradOps');

{ tanh: [Function],
  add: [Function],
  minus: [Function],
  dot: [Function],
  pow: [Function],
  mean: [Function] }

In [ ]:
GradOps.dot = (ret, nbA, nbB)=>{
  let nbs = [nbA, nbB];
  const nbA_grad = (ret, nbA)=>{
      let _sh = nbA.shape.map((d,i)=>i);
      let Taxis = [..._sh.slice(0,-2), ..._sh.slice(-1), ..._sh.slice(-2,-1)];
      let retGrad = Ops.dot(Ops.T(nbA, Taxis), ret); 
      return Ops.T(retGrad, Taxis);
  }
  
  const nbB_grad = (ret, nbB)=>{
      let _sh = nbA.shape.map((d,i)=>i);
      let Taxis = [..._sh.slice(0,-2), ..._sh.slice(-1), ..._sh.slice(-2,-1)];
      let retGrad = Ops.dot(Ops.T(nbA, Taxis), ret); 
      return retGrad;
  }

  const transformRet = ( ret, op_inputs )=>{
    let rS = ret.shape.slice();
    let nbA = op_inputs[0], nbB = op_inputs[1];
    if(rS.length <= 2){
        return ret;
    }
    else{
//         console.warn(op_inputs, nbA, nbB);
        let AShLen  = nbA.shape.length, BShLen = nbB.shape.length;
        let _selAxis = [AShLen>2?(AShLen-2):0, rS.length - 1];
        let [selAxis, newShape] = rS.reduce((ss,d,i)=>{
                 if(_selAxis.indexOf(i)==-1){ ss[0] = [...ss[0],i] }
                 else{ ss[1] = [...ss[1],d] }
                 return ss;
            },[[],[]])
        let newValue = new Float32Array(nd.getVolume(newShape));
        let reformedRet = Numb(newValue, newShape );  
        for(idx of nd.axisGenerator(selAxis, ret.shape)){
            reformedRet = Ops.add(reformedRet, ret.v[idx]) 
        }
//         console.warn(reformedRet)
        return reformedRet;
    }       
  }
  ret.transformRet = transformRet;
  ret.grad = nbs.map((nb, i)=>{
      const _bw  = nb.grad?nb:null;
      const _vjp = (i===0)?nbA_grad:nbB_grad;
      const _vid = 0;//nb.grad[0].vid;
      return { bw: _bw, vid: _vid, vjp: _vjp, op_inputs: nbs };
  });
  return ret;
}

In [2]:
Autograd.Operators = {};
for(let opName in Ops){
  const _OpFunc = Ops[opName];
  const appendGradOp = (...inputs)=>{
    let stopGrad = false;
    const [ last ] = inputs.slice(-1);
    if(typeof last === 'boolean'){
      stopGrad = last;//either true or false
      inputs = inputs.slice(0,-1);
    }
    let ret = _OpFunc(...inputs);
    if(stopGrad === true){
      return ret;
    }
    else{
      if(GradOps[opName] === undefined){
        throw Error(`gradOp[${opName}] Not implement`);
      }
      ret = GradOps[opName](ret, ...inputs);
      return ret;
    }
  }
  Autograd.Operators[opName] = appendGradOp
}

[Function: appendGradOp]

In [5]:

nd$grad = Autograd.grad(Autograd.Operators.dot)
            (nd$A.reshape([1,3,2]), nd$B.reshape([1,2,3]) );

	 [level] 1
	 [output] Float32Array [ 1, 1, 1, 1, 1, 1, 1, 1, 1 ]
	 [inputs] 0 Float32Array [ 0, 1, 2, 3, 4, 5 ]
	 [inputs] 0 Float32Array [ 0, 1, 2, 3, 4, 5 ]
	 [level] 2
	 [output] Float32Array [ 78, 105, 124, 166, 170, 227 ]
	 [level] 2
	 [output] Float32Array [ 78, 124, 170, 105, 166, 227 ]


[ numb {
    shape: [ 1, 3, 2 ],
    volume: 6,
    space: [ 6, 2, 1 ],
    value: Float32Array [ 78, 105, 124, 166, 170, 227 ],
    type: 'Numb',
    v: { v: [Object], sh: [Array], sp: [Array] },
    vid: 0 },
  numb {
    shape: [ 1, 2, 3 ],
    volume: 6,
    space: [ 6, 3, 1 ],
    value: Float32Array [ 78, 124, 170, 105, 166, 227 ],
    type: 'Numb',
    v: { v: [Object], sh: [Array], sp: [Array] },
    vid: 1 } ]

In [4]:
nd$A = Numb([0,1,2,3,4,5]);
nd$B = Numb([0,1,2,3,4,5]);

numb {
  shape: [ 6 ],
  volume: 6,
  space: [ 1 ],
  value: Float32Array [ 0, 1, 2, 3, 4, 5 ],
  type: 'Numb',
  v: { v: Float32Array [ 0, 1, 2, 3, 4, 5 ], sh: [ 6 ], sp: [ 1 ] } }

In [ ]:
_sh = [1,4]
sh = [..._sh.slice(0,-2), ..._sh.slice(-1), ..._sh.slice(-2,-1)];

In [7]:
Operators = Ops
Operators.dot = (nbA,nbB)=>{
  const _dot$1d = (sA, sB, nS)=>{
    let newShape = [1], newValue = new Float32Array(1);
    for(let v = 0; v < nS; v++){
      newValue[0] += nbA.value[v]*nbB.value[v];  
    }
    return Numb(newValue, newShape);
  }
  const _dot$2d = (sA, sB, nS, pA, pB)=>{
    let newShape = [ sA[0],sB[1] ];
    let newValue = new Float32Array( nd.getVolume(newShape) );
    const selector = newShape.map(d=>[0,d,1]);
    for(let px of nd.indexGenerator(selector, nd.getSpace(newShape) ) ){
      let aVx  = 0, bVx = 0, vx = px.vx, r = px.idx[0], c = px.idx[1];
      for(let v = 0; v < nS; v += 1){
        aVx = r*pA[0] + v;
        bVx = c + v*pB[0];
        newValue[vx] += nbA.value[aVx]*nbB.value[bVx];
      }
    } 
    return Numb(newValue, newShape);
  }
  const _checkShapeThenRun2 = (ndA, ndB)=>{
    const sA = nbA.shape, sB = nbB.shape;
    const pA = nbA.space, pB = nbB.space;
    if(sA.length == 1 && sB.length == 1){
      const nS = sA[0]===sB[0]?sA[0]:null;
      if(nS===null){ throw Error( 'shape not consitent' ) }  
      return _dot$1d(sA, sB, nS);
    }
    else if(sA.length == 2 && sA.length == 2){
      const nS = sA[1]===sB[0]?sA[1]:null;
      return _dot$2d(sA, sB, nS, pA, pB);
    }
    else if(sA.length == 1 && sB.length == 2)
    else if(sA.length > 2 && sB.length == 2){
      const sA$l = sA.length;
      const nS = sA[sA$l - 1]===sB[0]?sA[sA$l - 1]:null;
      if(nS===null){ throw Error( 'shape not consitent' ) }  
      let newShape = [ ...sA.slice(0,-1), sB[1] ];
      let newValue = new Float32Array( nd.getVolume(newShape) );
      let ret = Numb(newValue, newShape);   
      // console.log('newShape', newShape);
      let selector = sA.map((d,i)=>(i < (sA$l - 2))?[0,d,1]:d);
      let leftSelect;
      for(let px of nd.indexGenerator(selector, sA)){
        leftSelect = px.idx.slice(0,-2).join(',') + ',:,:';
        // console.warn('loop', leftSelect);
        // console.warn(ndA.v[leftSelect]);
        let _innerRet = Operators.dot(ndA.v[leftSelect], ndB);
        // console.warn(_innerRet);
        ret.v[leftSelect] = _innerRet;
      }
      return ret;
    }
    // else if(sA.length == 2 && sB.length == 2){
    //   const sA$l = sA.length;
    //   const nS = sA[sA$l - 1]===sB[0]?sA[sA$l - 1]:null;
    //   if(nS===null){ throw Error( 'shape not consitent' ) }  
    //   let newShape = [ ...sA.slice(0,-1), sB[1] ];
    //   let newValue = new Float32Array( nd.getVolume(newShape) );
    //   let ret = Numb(newValue, newShape);   
    //   // console.log('newShape', newShape);
    //   let selector = sA.map((d,i)=>(i < (sA$l - 2))?[0,d,1]:d);
    //   let leftSelect;
    //   for(let px of nd.indexGenerator(selector, sA)){
    //     leftSelect = px.idx.slice(0,-2).join(',') + ',:,:';
    //     // console.warn('loop', leftSelect);
    //     // console.warn(ndA.v[leftSelect]);
    //     let _innerRet = Operators.dot(ndA.v[leftSelect], ndB);
    //     // console.warn(_innerRet);
    //     ret.v[leftSelect] = _innerRet;
    //   }
    //   return ret;
    // }
    else if(sA.length > 2 && sB.length > 2){
      const sA$l = sA.length, sB$l = sB.length;
      const nS = sA[sA$l - 1]===sB[sB$l - 2]?sA[sA$l - 1]:null;
      if(nS===null){ throw Error( 'shape not consitent' ) }  
      let newShape = [  ...sA.slice(0,-1), 
                        ...sB.slice(0,-2), 
                        ...sB.slice(-1)   ];
      let newValue = new Float32Array( nd.getVolume(newShape) );
      let ret = Numb(newValue, newShape);   
      // console.log('newShape', newShape);
      let leftSelector  = sA.map((d,i)=>(i<(sA$l-2))?[0,d,1]:d);
      let rightSelector = sB.map((d,i)=>(i<(sB$l-2))?[0,d,1]:d);
      let leftSelect, rightSelect, select;
      for(let lpx of nd.indexGenerator(leftSelector, sA)){
        leftSelect = lpx.idx.slice(0,-2).join(',');
        for(let rpx of nd.indexGenerator(rightSelector, sB)){
          rightSelect = rpx.idx.slice(0,-2).join(',');
          select = leftSelect +',:,' + rightSelect + ',:';
          // console.warn('loop', leftSelect, rightSelect, select);  
          let _innerRet = Operators.dot(ndA.v[leftSelect], 
                                      ndB.v[rightSelect]);
          // console.warn(_innerRet);
          ret.v[select] = _innerRet;
        }
      }
      return ret;
    }
    else{
      throw Error( 'shape not consitent' );
    }
  }

  const _checkShapeThenRun = (nbA, nbB)=>{
    const sA = nbA.shape, sB = nbB.shape;
    const pA = nbA.space, pB = nbB.space;
    if(sA.length == 1 && sB.length == 1){
      const nS = sA[0]===sB[0]?sA[0]:null;
      if(nS===null){ throw Error( 'shape not consitent' ) }  
      return _dot$1d(sA, sB, nS);
    }
    else if(sA.length == 2 && sA.length == 2){
      const nS = sA[1]===sB[0]?sA[1]:null;
      return _dot$2d(sA, sB, nS, pA, pB);
    }
  }

  let ret = _checkShapeThenRun(nbA, nbB);
  return ret;
};

[Function]